### Praticando Reduces no LangGraph
___

Exercício Prático: 
--**Planejador de Viagens Parelelo**

    Vamos criar um grafo simples que para, um dada cidade, busca sugestões de restaurantes e de atrações turísticas em parelelo e as combina em um única lista de sugestões.

    1. Um NODE para buscar os restaurantes
    2. Um NODE para buscar atrações turísticas
    3. Uso do REDUCER para combinar em um única lista de sugestões
    4. Criação do STATE com pydantic, garantindo robustez e validação da estrutura de dados.

In [ ]:
# * 1. Importação das bibliotecas
import operator
import os
from typing import Annotated

from dotenv import find_dotenv, load_dotenv
from IPython.display import Image, display  # pyright: ignore[reportUnknownVariableType]
from langgraph.graph import END, START, StateGraph
from loguru import logger
from pydantic import BaseModel, Field

# * 2. Configuração do ambiente
_ = load_dotenv(find_dotenv())

In [ ]:
# 3. Configuração do LangSmith (Opcional, mas recomendado)
# Certifique-se de que as seguintes variáveis estão no seu .env:
# LANGCHAIN_TRACING_V2="true"
# LANGCHAIN_API_KEY="sua_api_key"
# LANGCHAIN_PROJECT="seu_nome_de_projeto" (ex: "LangGraph - Travel Planner")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = (
    "Exercício - Travel Planner"  # Você pode nomear como quiser
)

# 4. Configuração do Logger
logger.add("output.log", rotation="10 MB", level="INFO")
logger.info("Ambiente e bibliotecas carregados com sucesso.")

2025-09-19 11:41:42.572 | INFO     | __main__:<module>:11 - Ambiente e bibliotecas carregados com sucesso.


In [ ]:
class TravelState(BaseModel):
    """Representa o estado do grafo do planejador de viagens.

    Esta classe define a estrutura de dados que será passada entre os nós
    do grafo. Ela armazena a cidade de entrada e acumula as sugestões
    coletadas.

    Attributes:
        city (str): A cidade para a qual as sugestões serão buscadas.
        suggestions (list[str]): Uma lista que acumula sugestões de restaurantes
            e atrações. Utiliza um redutor (`operator.add`) para combinar
            resultados de nós que executam em paralelo.
    """

    city: str = Field(
        ..., description="A cidade para a qual as sugestões serão buscadas."
    )

    suggestions: Annotated[list[str], operator.add] = Field(
        default_factory=list, description="Lista combinada de restaurantes e atrações."
    )

Definição do NODE: `get_restaurants`

1. Criação do modelo de dados usados pela classe RestaurantSuggestions - que será usada no primeiro NODE, pela LLM

In [ ]:
# * Este modelo define a estrutura de saída da LLM que será usada para buscar restaurantes
class RestaurantSuggestions(BaseModel):
    """Modelo para sugestões de restaurantes.

    Este modelo define a estrutura de saída que será produzida pela LLM.
    Ele garante que sempre haverá uma lista fixa de restaurantes sugeridos,
    com no mínimo e no máximo três itens.

    Attributes:
        restaurants (list[str]): Lista de restaurantes sugeridos pela LLM.
            Deve conter exatamente 3 itens (min_items=3, max_items=3).
    """

    restaurants: list[str] = Field(
        ...,
        description="Lista de restaurantes sugeridos pela LLM",
        min_items=3,
        max_items=3,
    )

2. Criação da chain que será usada para gerar a sugestão de restaurantes.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# * 1. Instanciar o LLM com suporte a structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# * 2. Habilitar saída estruturada diretamente no LLM
llm_with_parser = llm.with_structured_output(RestaurantSuggestions)

# * 3. Criar o Prompt Template
prompt_restaurants = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente especialista em viagens e gastronomia. "
            "Responda com exatamente 3 restaurantes de qualidade.",
        ),
        ("user", "Sugira restaurantes na cidade de {city}."),
    ]
)

# * 4. Montar a chain final
chain_restaurants = prompt_restaurants | llm_with_parser

Criando o NODE que busca os restaurantes:

In [ ]:
def get_restaurants_node(state: TravelState) -> dict[str, list[str]]:
    """
    Executa uma chain para obter 3 sugestões de restaurantes para a cidade
    especificada no estado e as retorna para serem adicionadas à lista de sugestões.

    Args:
        state (TravelState): O estado atual do grafo, que contém a cidade.

    Returns:
        dict[str, list[str]]: Um dicionário cuja chave corresponde ao campo do
                               estado que deve ser atualizado. O valor é o dado
                               a ser enviado para o reducer.
    """
    # * 1. Extrai a informação necessária do estado de entrada.
    city = state.city
    logger.info(f"Iniciando busca de restaurantes para a cidade: {city}")

    # * 2. Invoca a chain que você criou.
    #    A entrada da chain é um dicionário cuja chave 'city' corresponde
    #    à variável {city} no seu prompt.
    suggestions_object = chain_restaurants.invoke({"city": city})

    # * 3. Extrai a lista de nomes do objeto Pydantic que a chain retornou.
    #    'suggestions_object' é uma instância da sua classe 'RestaurantSuggestions'.
    restaurant_names = suggestions_object.restaurants
    logger.info(f"Restaurantes encontrados: {restaurant_names}")

    # * 4. Retorna o resultado no formato que o LangGraph espera.
    #    - DEVE ser um dicionário.
    #    - A chave "suggestions" DEVE ser o nome exato do atributo no TravelState
    #      que você quer atualizar.
    #    - O valor (restaurant_names) será passado para o reducer (operator.add).
    return {"suggestions": restaurant_names}

Gerar o modelo de dados para lista de atrações, prompt, chain e por fim o node get_attractions_node.

In [ ]:
# --- Bloco de Código para o Nó de Atrações ---

# 1. Definição do Schema de Saída com Pydantic
class AttractionSuggestions(BaseModel):
    """Modelo para sugestões de atrações turísticas."""

    attractions: list[str] = Field(
        ...,
        description="Lista de atrações turísticas sugeridas pela LLM",
        min_items=3,
        max_items=3,
    )


# 2. Habilitar saída estruturada diretamente no LLM
# Presumindo que o 'llm' (ChatOpenAI) já foi instanciado na célula anterior
llm_with_parser_attractions = llm.with_structured_output(AttractionSuggestions)

# 3. Criar o Prompt Template específico para atrações
prompt_attractions = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente especialista em viagens e cultura. "
            "Responda com exatamente 3 atrações turísticas imperdíveis.",
        ),
        ("user", "Sugira atrações turísticas na cidade de {city}."),
    ]
)

# 4. Montar a chain final para atrações
chain_attractions = prompt_attractions | llm_with_parser_attractions


# 5. Função final do Nó para o LangGraph
def get_attractions_node(state: TravelState) -> dict[str, list[str]]:
    """
    Executa uma chain para obter 3 sugestões de atrações para a cidade
    e as retorna para serem adicionadas à lista de sugestões.
    """
    city = state.city
    logger.info(f"Iniciando busca de atrações para a cidade: {city}")

    # Invoca a chain de atrações
    suggestions_object = chain_attractions.invoke({"city": city})

    # Extrai a lista do objeto Pydantic
    attraction_names = suggestions_object.attractions
    logger.info(f"Atrações encontradas: {attraction_names}")

    # Retorna o dicionário com a MESMA chave 'suggestions' para o reducer
    return {"suggestions": attraction_names}

Gerar o grafo e usar o reducer para juntar as saídas em paralelo

In [ ]:
# --- Bloco de Montagem do Grafo ---

# 1. Instanciar o Grafo
# Criamos o "construtor" do grafo, associando-o ao nosso modelo de estado TravelState.
# Vamos usar a variável 'graph' para o construtor, como é comum.
graph = StateGraph(TravelState)
logger.info("StateGraph instanciado com o estado TravelState.")

# 2. Adicionar os Nós
# Damos um nome (string) para cada nó e o associamos à sua respectiva função.
graph.add_node("restaurants", get_restaurants_node)
graph.add_node("attractions", get_attractions_node)
logger.info("Nós 'restaurants' e 'attractions' adicionados ao workflow.")

# 3. Definir as Arestas (Edges) para Execução Paralela
# A partir do início (START), o fluxo é direcionado para AMBOS os nós.
# Isso informa ao LangGraph para executá-los em paralelo.
graph.add_edge(START, "restaurants")
graph.add_edge(START, "attractions")

# Após cada nó terminar sua execução, o grafo pode chegar ao fim (END).
graph.add_edge("restaurants", END)
graph.add_edge("attractions", END)
logger.info(
    "Arestas definidas para execução paralela do START para ambos os nós, e de ambos para o END."
)

# 4. Compilar o Grafo
# O método .compile() transforma nossa definição em um objeto executável.
# Usando a variável final 'graph' como você sugeriu.
graph = graph.compile()

print("Grafo compilado com sucesso e pronto para ser executado!")
logger.info("Grafo compilado com sucesso e atribuído à variável 'graph'.")

2025-09-19 11:41:43.425 | INFO     | __main__:<module>:7 - StateGraph instanciado com o estado TravelState.
2025-09-19 11:41:43.427 | INFO     | __main__:<module>:13 - Nós 'restaurants' e 'attractions' adicionados ao workflow.
2025-09-19 11:41:43.428 | INFO     | __main__:<module>:24 - Arestas definidas para execução paralela do START para ambos os nós, e de ambos para o END.
2025-09-19 11:41:43.431 | INFO     | __main__:<module>:32 - Grafo compilado com sucesso e atribuído à variável 'graph'.


Grafo compilado com sucesso e pronto para ser executado!


In [ ]:
# --- Bloco de Execução e Visualização ---

# 1. Definir o input para o grafo
# A chave 'city' deve corresponder ao atributo no nosso TravelState.
cidade_para_pesquisar = "Alto Paraíso"
input_data = {"city": cidade_para_pesquisar}
logger.info(f"Iniciando a invocação do grafo para a cidade: {cidade_para_pesquisar}")

# 2. Invocar o grafo com os dados de entrada
# O LangSmith irá capturar este traço para podermos depurar e observar.
final_result = graph.invoke(input_data)
logger.info("Grafo executado com sucesso.")

# 3. Imprimir o resultado final e combinado
print("--- RESULTADO FINAL COMBINADO ---")
print(f"Sugestões para {final_result['city']}:")
# O 'final_result' é o estado final, e nossa lista combinada está na chave 'suggestions'.
for suggestion in final_result["suggestions"]:
    print(f"- {suggestion}")

# 4. Gerar e exibir a imagem do grafo
print("\n--- ESTRUTURA DO GRAFO (PNG) ---")
try:
    # O método .get_graph() nos dá acesso à estrutura,
    # e .draw_mermaid_png() a renderiza como uma imagem.
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(
        f"Não foi possível gerar a imagem do grafo. Verifique as dependências (ex: pygraphviz). Erro: {e}"
    )

2025-09-19 11:41:43.440 | INFO     | __main__:<module>:7 - Iniciando a invocação do grafo para a cidade: Alto Paraíso
2025-09-19 11:41:43.633 | INFO     | __main__:get_attractions_node:42 - Iniciando busca de atrações para a cidade: Alto Paraíso
2025-09-19 11:41:43.633 | INFO     | __main__:get_restaurants_node:16 - Iniciando busca de restaurantes para a cidade: Alto Paraíso
2025-09-19 11:41:45.688 | INFO     | __main__:get_restaurants_node:26 - Restaurantes encontrados: ['Café do Cerrado', "Restaurante Roda D'Água", 'Vila do Cerrado']
2025-09-19 11:41:45.699 | INFO     | __main__:get_attractions_node:49 - Atrações encontradas: ['Cataratas dos Couros', 'Parque Nacional da Chapada dos Veadeiros', 'Véu de Noiva e o Morro da Baleia']
2025-09-19 11:41:45.702 | INFO     | __main__:<module>:12 - Grafo executado com sucesso.


--- RESULTADO FINAL COMBINADO ---
Sugestões para Alto Paraíso:
- Cataratas dos Couros
- Parque Nacional da Chapada dos Veadeiros
- Véu de Noiva e o Morro da Baleia
- Café do Cerrado
- Restaurante Roda D'Água
- Vila do Cerrado

--- ESTRUTURA DO GRAFO (PNG) ---
Não foi possível gerar a imagem do grafo. Verifique as dependências (ex: pygraphviz). Erro: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


![Diagrama do Grafo com Reducer](/home/fabiolima/Workdir/langchain/study_langchain/notebooks/reducer_langgraph_diagram.png "Grafo com Reducer")

---

### Conclusão e Resumo do Exercício

Este notebook demonstrou a construção de um grafo computacional paralelo utilizando LangGraph para resolver um problema prático: a criação de um planejador de viagens. O objetivo principal era buscar, de forma simultânea, sugestões de restaurantes e atrações turísticas para uma cidade e, em seguida, combinar os resultados em uma única lista coesa.

A seguir, detalhamos o passo a passo da implementação.

#### Passo 1: Preparação do Ambiente
A base de qualquer projeto robusto é um ambiente bem configurado. Iniciamos importando todas as bibliotecas necessárias, incluindo `langgraph`, `pydantic`, `langchain_openai`, `loguru` para logs detalhados e `dotenv` para o gerenciamento de chaves de API. Também configuramos o `LangSmith` para garantir a observabilidade e o rastreamento da execução do nosso grafo, o que é fundamental para a depuração.

#### Passo 2: Design Centrado no Estado (State-First)
A "memória" ou o "estado" do nosso grafo foi definida usando a classe `TravelState`, que herda de `pydantic.BaseModel` para garantir a validação e a clareza dos dados. O componente mais crítico desta classe foi o atributo `suggestions`. Ao tipá-lo como `Annotated[list[str], operator.add]`, instruímos o LangGraph a usar a função `operator.add` como um **reducer**. Isso significa que, quando múltiplos nós tentarem escrever nesta lista ao mesmo tempo, seus resultados serão concatenados em vez de sobrescritos ou gerarem um erro.

#### Passo 3: Construção dos Nós Inteligentes
Cada tarefa principal foi encapsulada em um "nó" do grafo. Construímos dois nós: `get_restaurants_node` e `get_attractions_node`. Para garantir que a saída de cada nó fosse confiável e bem estruturada, seguimos um padrão moderno e robusto:
1.  **Definição de um Schema de Saída:** Para cada nó, criamos uma classe Pydantic (`RestaurantSuggestions` e `AttractionSuggestions`) que define a estrutura exata da resposta que esperamos do LLM (uma lista com 3 strings).
2.  **Saída Estruturada:** Utilizamos o método `.with_structured_output()` no nosso LLM (`ChatOpenAI`). Esta é a prática recomendada para forçar o modelo a retornar um JSON que adere perfeitamente ao nosso schema Pydantic.
3.  **Criação de uma Chain:** Combinamos o prompt, o LLM com parser e a lógica de chamada em uma `chain` da LangChain Expression Language (LCEL).
4.  **Função do Nó:** A função do nó atuou como uma "ponte", recebendo o estado do grafo, extraindo a cidade, invocando a `chain` e retornando um dicionário com a chave `"suggestions"`, garantindo que o resultado fosse direcionado para o reducer.

#### Passo 4: Montagem e Execução do Grafo Paralelo
Com os nós prontos, montamos o grafo:
* Adicionamos os dois nós (`restaurants` e `attractions`) ao nosso `StateGraph`.
* A execução paralela foi definida ao criarmos duas **arestas (edges)** a partir do ponto de entrada `START`, uma para cada nó (`START -> restaurants` e `START -> attractions`).
* Conectamos ambos os nós ao ponto de finalização `END`.
* Finalmente, compilamos o grafo e o invocamos com uma cidade. O resultado, impresso na tela, foi uma lista única contendo 6 sugestões (3 restaurantes e 3 atrações), validando que a execução paralela e a combinação com o reducer foram um sucesso.

#### Principais Aprendizados
* **Paralelismo em LangGraph:** Como estruturar um fluxo de trabalho onde múltiplas tarefas ocorrem simultaneamente.
* **Reducers para Gerenciamento de Estado:** O uso de `Annotated` para resolver conflitos de escrita e agregar dados de forma inteligente.
* **Robustez com Pydantic:** A utilização de Pydantic tanto para definir o estado do grafo quanto para garantir a saída estruturada e validada de LLMs.
* **Observabilidade:** A importância de ferramentas como `loguru` e `LangSmith` para entender e depurar o comportamento do grafo.
* **Tipos de dados no Reducers:** Todos os nós que escrevem na mesma chave de estado com um reducer devem retornar o mesmo tipo de dado que a função redutora espera para poder operar corretamente.

# TODO 
1. Acrescentar mais 2 nós, mudar o graph.
2. Adicionar e modificar com o código a seguir `down below code`

In [ ]:
# --- Bloco de Código para os Novos Nós: Cultural e Contemplativo ---

# 1a. Schema Pydantic para Passeios Culturais
class CulturalSuggestions(BaseModel):
    """Modelo para sugestões de passeios culturais."""

    cultural_tours: list[str] = Field(
        ...,
        description="Lista de 2 sugestões de passeios culturais.",
        min_length=2,
        max_length=2,
    )


# 1b. Schema Pydantic para Passeios Contemplativos
class ContemplativeSuggestions(BaseModel):
    """Modelo para sugestões de passeios contemplativos."""

    contemplative_tours: list[str] = Field(
        ...,
        description="Lista de 2 sugestões de passeios contemplativos.",
        min_length=2,
        max_length=2,
    )


# Presumindo que 'llm' já foi instanciado
# 2a. Chain para Passeios Culturais
llm_with_parser_cultural = llm.with_structured_output(CulturalSuggestions)
prompt_cultural = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um guia turístico especializado na cultura e história local. Sugira 2 atividades culturais autênticas.",
        ),
        ("user", "Quais são os melhores passeios culturais na cidade de {city}?"),
    ]
)
chain_cultural = prompt_cultural | llm_with_parser_cultural

# 2b. Chain para Passeios Contemplativos
llm_with_parser_contemplative = llm.with_structured_output(ContemplativeSuggestions)
prompt_contemplative = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um guia de ecoturismo e bem-estar. Sugira 2 passeios focados em contemplação da natureza, como ver o pôr do sol, nascer da lua ou observação de pássaros.",
        ),
        ("user", "Quais são os melhores passeios contemplativos na cidade de {city}?"),
    ]
)
chain_contemplative = prompt_contemplative | llm_with_parser_contemplative


# 3a. Função do Nó Cultural
def get_cultural_node(state: TravelState) -> dict[str, list[str]]:
    city = state.city
    logger.info(f"Iniciando busca de passeios CULTURAIS para: {city}")
    suggestions = chain_cultural.invoke({"city": city})
    logger.info(f"Passeios culturais encontrados: {suggestions.cultural_tours}")
    return {"suggestions": suggestions.cultural_tours}


# 3b. Função do Nó Contemplativo
def get_contemplative_node(state: TravelState) -> dict[str, list[str]]:
    city = state.city
    logger.info(f"Iniciando busca de passeios CONTEMPLATIVOS para: {city}")
    suggestions = chain_contemplative.invoke({"city": city})
    logger.info(
        f"Passeios contemplativos encontrados: {suggestions.contemplative_tours}"
    )
    return {"suggestions": suggestions.contemplative_tours}

In [ ]:
# --- Bloco de Montagem do Grafo ATUALIZADO (4 nós paralelos) ---

# 1. Instanciar o Grafo
graph_v2 = StateGraph(TravelState)

# 2. Adicionar TODOS os quatro nós
graph_v2.add_node("restaurants", get_restaurants_node)
graph_v2.add_node("attractions", get_attractions_node)
graph_v2.add_node("cultural", get_cultural_node)
graph_v2.add_node("contemplative", get_contemplative_node)

# 3. Definir as Arestas para Execução Paralela
# O START agora dispara QUATRO nós ao mesmo tempo!
graph_v2.add_edge(START, "restaurants")
graph_v2.add_edge(START, "attractions")
graph_v2.add_edge(START, "cultural")
graph_v2.add_edge(START, "contemplative")

# 4. Conectar todos os nós ao FIM
graph_v2.add_edge("restaurants", END)
graph_v2.add_edge("attractions", END)
graph_v2.add_edge("cultural", END)
graph_v2.add_edge("contemplative", END)

# 5. Compilar o novo grafo
compiled_graph_v2 = graph_v2.compile()

print("Grafo V2 com 4 nós paralelos compilado com sucesso!")

In [ ]:
TODO 

6. Conceitos-chave destacados
Fluxos de trabalho sequenciais são simples porque cada nó modifica o estado um após o outro.
Fluxos de trabalho paralelos exigem reducers para evitar conflitos de atualização.
Reducers definem como mesclar atualizações simultâneas:
- Somar números → Soma
- Somar listas → Concatenação
- Somar mensagens → Agregação de mensagens
MessageState simplifica o uso de mensagens do LangChain em fluxos de trabalho sem a necessidade de esquemas personalizados.

7. Conclusão
Reducers são essenciais para uma execução paralela segura e correta no LangGraph.
Eles permitem que múltiplos nós contribuam para o mesmo campo sem perda de dados ou colisões.
Projetar corretamente o estado e os reducers possibilita fluxos de trabalho em grafo escaláveis e modulares.
